In [2]:
#Importing JSON and MongoClient Library
import json
from pymongo import MongoClient
import os

In [3]:
#Establishing Connection with Mongo Server and Creating Database and Assigning Collection Name
client = MongoClient('localhost', 27017)
db = client['COVID']
collection_covid = db['COVID_HOSPITAL']

In [5]:
#Opening the COVID JSON File and reading the data and storing the data within the defined Collection Name
os.chdir('/Users/adityaraj/Desktop/Database and Analytics Programming Folder/CA Project/')
with open('COVID-19 Reported Patient Impact and Hospital Capacity by State Timeseries.json') as f:
    file_data = json.load(f)

collection_covid.insert_one(file_data)
client.close()

In [ ]:
#Extracting Column Names from the Columns tag within JSON file and storing into a 'cols' list

getData = pd.DataFrame(list(collection_covid.find())) 
cols = []

for data in getData["meta"]:
    for column in data['view']['columns']:
        columnName = column['name']
        if columnName not in cols:
            cols.append(columnName)
        
print(cols)